# Projeto: Rastreamento de Experimentos com MLflow e Azure ML

Este notebook demonstra uma capacidade fundamental de MLOps: o **rastreamento de experimentos** de machine learning. 

Utilizando a biblioteca open-source **MLflow** integrada ao **Azure Machine Learning**, este projeto mostra como registrar, monitorar e organizar execuções de treinamento de modelos.

**Objetivo:**
O objetivo não é apenas treinar um modelo, mas criar um processo **reprodutível** e **auditável**, onde cada parâmetro, métrica e artefato (como o próprio modelo) é salvo e versionado.

**Etapas:**
1.  Conexão ao Workspace do Azure ML.
2.  Preparação dos dados de diabetes.
3.  Configuração do Experimento MLflow.
4.  Execução 1: Treinamento com **Autologging**.
5.  Execução 2: Treinamento com **Rastreamento Manual** (Parâmetros e Métricas).
6.  Execução 3: Comparação de Hiperparâmetros.
7.  Execução 4: Comparação de Modelos (Árvore de Decisão).
8.  Execução 5: Rastreamento de Artefatos (Plot da Curva ROC).

## 1. Configuração e Conexão ao Workspace

Primeiro, importamos todas as bibliotecas necessárias e nos conectamos ao Workspace do Azure ML usando o SDK v2.

In [ ]:
# Imports do Azure ML
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

# Import do MLflow
import mlflow
import mlflow.sklearn

# Bibliotecas de Data Science e Modelagem
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

print("Bibliotecas importadas com sucesso.")

In [ ]:
try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

# Carrega a configuração do workspace (do config.json)
ml_client = MLClient.from_config(credential=credential)

print(f"Conectado ao workspace: {ml_client.workspace_name}")

## 2. Preparação dos Dados

Carregamos o dataset de diabetes, separamos as *features* (X) e o *label* (y), e dividimos em conjuntos de treino e teste.

In [ ]:
# Carregar os dados
print("Carregando dados...")
df = pd.read_csv('./data/diabetes.csv')

# Separar features e label
X, y = df[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, df['Diabetic'].values

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print(f"Dados prontos: {X_train.shape[0]} amostras de treino, {X_test.shape[0]} amostras de teste.")

## 3. Configuração do Experimento MLflow

Antes de treinar, definimos um **Experimento** no MLflow. Isso agrupará todas as nossas execuções (runs) relacionadas sob um único nome no Azure ML Studio, facilitando a comparação.

In [ ]:
experiment_name = "projeto-mlflow-diabetes"
mlflow.set_experiment(experiment_name)

print(f"Experimento '{experiment_name}' configurado.")

## 4. Execução 1: Treinamento com Autologging

O MLflow oferece uma funcionalidade de `autolog()` que rastreia automaticamente parâmetros, métricas e o modelo com apenas uma linha de código. É ótimo para prototipagem rápida.

In [ ]:
# O Autologging rastreia tudo automaticamente
print("Iniciando execução com Autologging...")
with mlflow.start_run():
    mlflow.sklearn.autolog()
    
    # Treina o modelo
    model = LogisticRegression(C=1/0.1, solver="liblinear").fit(X_train, y_train)
    
print("Execução com Autologging concluída.")

## 5. Execução 2: Treinamento com Rastreamento Manual

O Autologging é conveniente, mas às vezes queremos controle total sobre o que é registrado. Vamos desabilitar o autologging e registrar manualmente um parâmetro (`regularization_rate`) e uma métrica (`Accuracy`).

In [ ]:
# Desabilita o autologging para as próximas execuções
mlflow.sklearn.autolog(disable=True)

print("Iniciando execução com Rastreamento Manual...")
with mlflow.start_run():
    # Treina o modelo
    reg_rate = 0.1
    model = LogisticRegression(C=1/reg_rate, solver="liblinear").fit(X_train, y_train)

    # Avalia o modelo
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)

    # Rastreamento Manual
    mlflow.log_param("regularization_rate", reg_rate)
    mlflow.log_metric("Accuracy", acc)
    
    print(f"Execução manual concluída. Accuracy: {acc:.4f}")

## 6. Execução 3: Comparação de Hiperparâmetros

O principal uso do MLflow é comparar experimentos. Vamos treinar o mesmo modelo, mas com um hiperparâmetro diferente (taxa de regularização = 0.01) e registrar os resultados.

In [ ]:
print("Iniciando Execução 3 (Hiperparâmetro diferente)...")
with mlflow.start_run():
    # Treina o modelo com um novo hiperparâmetro
    reg_rate = 0.01
    model = LogisticRegression(C=1/reg_rate, solver="liblinear").fit(X_train, y_train)

    # Avalia o modelo
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)

    # Rastreamento Manual
    mlflow.log_param("regularization_rate", reg_rate)
    mlflow.log_metric("Accuracy", acc)
    
    print(f"Execução 3 concluída. Accuracy: {acc:.4f}")

## 7. Execução 4: Comparação de Modelos (Árvore de Decisão)

Agora, vamos comparar um algoritmo completamente diferente. Treinaremos uma `DecisionTreeClassifier` e registraremos sua acurácia.

In [ ]:
print("Iniciando Execução 4 (Estimador diferente)...")
with mlflow.start_run():
    # Treina um modelo diferente
    model = DecisionTreeClassifier().fit(X_train, y_train)

    # Avalia o modelo
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)

    # Rastreamento Manual
    mlflow.log_param("estimator_name", "DecisionTreeClassifier")
    mlflow.log_metric("Accuracy", acc)
    
    print(f"Execução 4 concluída. Accuracy: {acc:.4f}")

## 8. Execução 5: Rastreamento de Artefatos (Curva ROC)

O MLflow não serve apenas para números, ele também pode salvar arquivos, chamados **Artefatos**. Vamos treinar o modelo de Árvore de Decisão novamente, mas desta vez vamos gerar um gráfico da **Curva ROC** e salvá-lo como uma imagem (`.png`) junto com a execução.

In [ ]:
print("Iniciando Execução 5 (Com Artefato)...")
with mlflow.start_run():
    # Treina o modelo
    model = DecisionTreeClassifier().fit(X_train, y_train)

    # Avalia o modelo
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)

    # --- Criação do Artefato (Gráfico ROC) ---
    y_scores = model.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
    
    fig = plt.figure(figsize=(6, 4))
    plt.plot([0, 1], [0, 1], 'k--') # Linha diagonal
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    
    # Salva a figura localmente
    plot_filename = "ROC-Curve.png"
    plt.savefig(plot_filename)
    plt.close(fig) # Fecha a figura para não exibir no notebook
    
    # Rastreamento Manual
    mlflow.log_param("estimator_name", "DecisionTreeClassifier")
    mlflow.log_metric("Accuracy", acc)
    
    # Log do Artefato
    mlflow.log_artifact(plot_filename)
    
    print(f"Execução 5 concluída. Artefato 'ROC-Curve.png' registrado.")

## 9. Conclusão e Próximos Passos

Todos os 5 experimentos foram registrados com sucesso!

Para ver os resultados, vá para o seu **Workspace do Azure ML Studio**:
1.  Clique na aba **"Trabalhos" (Jobs)**.
2.  Clique no experimento chamado **`projeto-mlflow-diabetes`**.
3.  Você verá todas as 5 execuções (runs) que acabamos de fazer.
4.  Selecione várias e clique em **"Comparar"** para ver as métricas e parâmetros lado a lado.
5.  Na "Execução 5", clique nela, vá para **"Saídas + logs"** e você encontrará o artefato `ROC-Curve.png` salvo.

Temos agora um histórico completo e auditável dos nossos experimentos, permitindo total reprodutibilidade.